# Word Embedding
---


## Import Libraries

In [1]:
# Import
from nltk.tokenize import word_tokenize
from gensim.models.phrases import Phrases, Phraser
#import fuzzysearch
#import fuzzywuzzy
import pandas as pd
import PyPDF2

## Read Text from PDF

In [2]:
pdfFileObj = open('20221118-FULL.pdf', 'rb') #Create pdf object
pdfReader = PyPDF2.PdfReader(pdfFileObj) 

allText = ""
for i in range(792): #Iterating over n pages 
    pageObj = pdfReader.pages[i]
    page = pageObj.extract_text()
    allText += page #Store page content into string variable

allText;

## Data Preprocessing 

In [3]:
# Preprocessing
allText = allText.replace(';','')

alltext_string_all = allText.replace('Legal basis',';Legal basis').replace('Identity information',';Identity information').replace('Programme', ';Programme').replace('Birth information',';Birth information').replace('Citizenship information',';Citizenship information').replace('Contact information',';Contact information')
testListSplit = alltext_string_all.split('reference number: EU')
alltext_string_all[:1000]


'EUROPEAN COMMISSION\nFinancial Stability, Financial Services and Capital Markets Union (DG FISMA)\nEuropean Union\nConsolidated Financial Sanctions List\nThis list has been updated on 18/11/2022 11:37European Union Consolidated Financial Sanctions List\nTable of contents\n1. Introduction3 \n2. Individuals or persons3 \n3. Entities or groups631 \n4. Disclaimer792 \nPage 2 on 792European Union Consolidated Financial Sanctions List\n1. INTRODUCTION\nThe present document contains the Consolidated List of persons, groups and entities subject, under EU Sanctions, to an asset freeze and the prohibition to\nmake funds and economic resources available to them. The latest version of this file is here.\n2. INDIVIDUALS OR PERSONS\nEU reference number: EU.1787.1\n;Legal basis: 2017/404 (OJ L63)\n;Programme: AFG - Afghanistan\n;Identity information:\n• Name/Alias: Sirajuddin Jallaloudine Haqqani  Function: Na’ib Amir (Deputy Commander)  \n• Name/Alias: Khalifa  \n• Name/Alias: Saraj Haqani  \n• Nam

In [4]:
# Create dataframe
dbn_df = pd.DataFrame([sub.split(";") for sub in testListSplit])
dbn_df.fillna("",inplace=True)
dbn_df.columns = ['EU Reference', 'Legal basis', 'Programme', 'Identity information', 'Birth information', 'Citizenship information', 'Contact information']
#,'','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','']

In [5]:
dbn_df


,EU Reference,Legal basis,Programme,Identity information,Birth information,Citizenship information,Contact information
0,"EUROPEAN COMMISSION\nFinancial Stability, Fina...",,,,,,
1,.1787.1\n,Legal basis: 2017/404 (OJ L63)\n,Programme: AFG - Afghanistan\n,Identity information:\n• Name/Alias: Sirajuddi...,Birth information:\n• Birth date: Circa from 1...,Citizenship information:\n• Citizenship: Afgha...,"Contact information: \n• Address: Pakistan, Mi..."
2,.2272.60\n,Legal basis: 2017/404 (OJ L63)\n,Programme: AFG - Afghanistan\n,Identity information:\n• Name/Alias: Nasiruddi...,Birth information:\n• Birth date: Circa from 1...,Citizenship information:\n• Citizenship: Afgha...,Contact information: \n• Address: Pakistan \n...
3,.2274.87\n,Legal basis: 2017/404 (OJ L63)\n,Programme: AFG - Afghanistan\n,Identity information:\n• Name/Alias: Gul Agha ...,Birth information:\n• Birth date: Circa 1972 ...,Contact information: \n• Address: Pakistan \n...,
4,.2291.0\n,Legal basis: 2017/404 (OJ L63)\n,Programme: AFG - Afghanistan\n,Identity information:\n• Name/Alias: Amir Abdu...,Birth information:\n• Birth date: Circa 1972 ...,Citizenship information:\n• Citizenship: Afgha...,"Contact information: \n• Address: Pakistan, Ka..."
...,...,...,...,...,...,...,...
3635,.9027.6\n,Legal basis: 2022/2229 (OJ L293-I)\n,Programme: UKR - Ukraine\nCompany name:\n• Nam...,,,,
3636,.9028.5\n,Legal basis: 2022/2229 (OJ L293-I)\n,Programme: UKR - Ukraine\nCompany name:\n• Nam...,,,,
3637,.5121.69\n,Legal basis: UNLI - 14.05.2019\n,Programme: UNLI - UN Listing\nPage 790 on 792E...,,,,
3638,.7817.36\n,Legal basis: 2022/419 (OJ L86)\n,Programme: YEM - Yemen\nCompany name:\n• Name:...,,,,


In [6]:
def cleanupDataframe(dataframe):
    dataframe['Legal basis'] = dataframe['Legal basis'].apply(lambda x: x.replace('Legal basis: ','').replace('\n',''))
    dataframe['Programme'] = dataframe['Programme'].apply(lambda x: x.replace('Programme: ','').replace('\n',''))
    dataframe['Identity information'] = dataframe['Identity information'].apply(lambda x: x.replace('Identity information:','').replace('Name/Alias:',';').replace('•','').replace('Function:','').split(';'))
    dataframe.replace('^\s+', '', regex=True, inplace=True) #front
    dataframe.replace('\s+$', '', regex=True, inplace=True) #end
    dataframe['Birth information'] = dataframe['Birth information'].apply(lambda x: x.replace('Birth information:\n•',''))
    return dataframe

def stripDataframe(dataframe):
    dataframe['Identity information'] = dataframe['Identity information'].apply(lambda x: [y.strip() for y in x ])
    return dataframe


# def listCreation(dataframe):
#     identityInformation = []
#     for i in dataframe:
#         temp = dataframe.tolist()
#         identityInformation.append(temp)
#     return identityInformation

In [7]:
cleanupDataframe(dbn_df)
stripDataframe(dbn_df)

,EU Reference,Legal basis,Programme,Identity information,Birth information,Citizenship information,Contact information
0,"EUROPEAN COMMISSION\nFinancial Stability, Fina...",,,[],,,
1,.1787.1,2017/404 (OJ L63),AFG - Afghanistan,"[, Sirajuddin Jallaloudine Haqqani Na’ib Ami...",Birth date: Circa from 1977 to 1978 Birth pl...,Citizenship information:\n• Citizenship: Afgha...,"Contact information: \n• Address: Pakistan, Mi..."
2,.2272.60,2017/404 (OJ L63),AFG - Afghanistan,"[, Nasiruddin Haqqani A leader of the Haqqan...",Birth date: Circa from 1970 to 1973 Birth pl...,Citizenship information:\n• Citizenship: Afgha...,Contact information: \n• Address: Pakistan \n...
3,.2274.87,2017/404 (OJ L63),AFG - Afghanistan,"[, Gul Agha Ishakzai, Mullah Gul Agha, Mullah ...",Birth date: Circa 1972 Birth place: Afghanis...,Contact information: \n• Address: Pakistan \n...,
4,.2291.0,2017/404 (OJ L63),AFG - Afghanistan,"[, Amir Abdullah Former Kandahar Province De...",Birth date: Circa 1972 Birth place: Afghanis...,Citizenship information:\n• Citizenship: Afgha...,"Contact information: \n• Address: Pakistan, Ka..."
...,...,...,...,...,...,...,...
3635,.9027.6,2022/2229 (OJ L293-I),UKR - UkraineCompany name:• Name: Islāma revol...,[],,,
3636,.9028.5,2022/2229 (OJ L293-I),UKR - UkraineCompany name:• Name: صنایع هوایی ...,[],,,
3637,.5121.69,UNLI - 14.05.2019,UNLI - UN ListingPage 790 on 792European Union...,[],,,
3638,.7817.36,2022/419 (OJ L86),YEM - YemenCompany name:• Name: ANSARALLAH• Na...,[],,,


In [8]:
# #dbn_df["Identity information"][1].split(';')
# cleanelement = []
# for element in dbn_df["Identity information"][1].split(';'): 
#     cleanelement.append(element.strip())
#     #element.strip()
# print(cleanelement)

In [9]:
dbn_df.to_excel('NameCorpusPreparation.xlsx')

In [10]:
dbn_df["Contact information"][4]

'Contact information: \n• Address: Pakistan, Karachi  \nRemark: Believed to be in Afghanistan/Pakistan border area. INTERPOL-UN Security Council Special Notice web link: https://www.interpol. int/en/\nnotice/search/un/1621271\nEU'

In [11]:
type(dbn_df['Identity information'][0])

list

In [12]:
#print(dbn_df[1])

KeyError: 1

### Split into Persons Dataframe and Company Dataframe

In [13]:
print(dbn_df.columns.tolist())

['EU Reference', 'Legal basis', 'Programme', 'Identity information', 'Birth information', 'Citizenship information', 'Contact information']


## Fuzzy string matching
---

In [14]:
from thefuzz import fuzz, process 

# Preprocessing
n = dbn_df['Identity information'].to_string().split()
n

['0',
 '[]',
 '1',
 '[,',
 'Sirajuddin',
 'Jallaloudine',
 'Haqqani',
 'Na’ib',
 'Ami...',
 '2',
 '[,',
 'Nasiruddin',
 'Haqqani',
 'A',
 'leader',
 'of',
 'the',
 'Haqqan...',
 '3',
 '[,',
 'Gul',
 'Agha',
 'Ishakzai,',
 'Mullah',
 'Gul',
 'Agha,',
 'Mullah',
 '...',
 '4',
 '[,',
 'Amir',
 'Abdullah',
 'Former',
 'Kandahar',
 'Province',
 'De...',
 '5',
 '[,',
 'Agha',
 'Jan',
 'Alizai,',
 'Haji',
 'Agha',
 'JanAlizai,',
 'Abdul...',
 '6',
 '[,',
 'Khalil',
 'Ahmed',
 'Haqqani',
 'Title:',
 'Haji,',
 'Khaleel',
 '...',
 '7',
 '[,',
 'Saleh',
 'Mohammad',
 'Kakar',
 'Akhtar',
 'Muhammad,',
 'Saleh...',
 '8',
 '[,',
 'Hamidullah',
 'Akhund',
 'Title:',
 'Mullah',
 'Head',
 'of',
 '...',
 '9',
 '[,',
 'Pahlawan',
 'Shamsuddin,',
 'Shamsuddin',
 'Title:',
 '(a)...',
 '10',
 '[,',
 'Ezatullah',
 'Haqqani',
 'Khan',
 'Sayyid',
 'Title:',
 'Maula...',
 '11',
 '[,',
 'Aminullah',
 'Amin',
 'Quddus',
 'Title:',
 'Maulavi',
 'Gov...',
 '12',
 '[,',
 'Mullah',
 'Jabar,',
 'Muawin',
 'Jabbar,',
 

In [15]:
#name = 'Pascal Breucker'


# Search function
def findBusinessPartner(input, name, threshold):
    namelist = name.split(" ")
    print(namelist)
    count = 0
    resultlist = []
    for name in namelist:

        for i in input:
        
            if fuzz.ratio(i, name) >= threshold:
                count += 1
                #print(fuzz.ratio(i, name), name, i)
                resultlist.append([fuzz.ratio(i,name) ,name,i])
                
    resultlist.sort(key = lambda x: x[0], reverse = True)
    if count == 0: 
        print(f'Your requested Business partner {name} with fuzzy threshold {threshold} cannot be found on the EU Sanctions List')
    
    return resultlist


# to-do: fuzz.ratio selbst coden 

In [16]:
#name = 'siray Qusay'
name = input('gib dein scheiß hier ein')
findBusinessPartner(n, name, 60)

['test']


[[67, 'test', 'Mesut'],
 [60, 'test', 'Mateso'],
 [60, 'test', 'Ernest'],
 [60, 'test', 'Modest']]

## Custom - Word2Vec
---

- Neuen Input Vektorisieren

In [17]:
import gensim 
from gensim.models import Word2Vec

In [18]:
#vocabulary preprocessing
vocabulary = dbn_df['Identity information'].tolist()
vocabulary

# Creating Word2Vec
model = Word2Vec(
    sentences = vocabulary, min_count=1, vector_size= 5, workers=3, window =3, sg = 1
)

In [55]:
# new User input to vocabulary corpus
traininput = input("")
vocabulary_copy = vocabulary[:]
vocabulary_copy.append([traininput])

# create w2v Model for the vocabulary copy
test_model = Word2Vec(
    sentences = vocabulary_copy, min_count=1, vector_size= 10, workers=3, window =3, sg = 1
)

In [56]:
print(test_model.wv[traininput])
#vocabulary

[ 0.03398594  0.05184268  0.06365552 -0.02589758  0.07231338  0.02719566
  0.02934089 -0.02278296 -0.03107939 -0.02498434]


In [57]:
test_model.wv.most_similar([traininput], topn=5)
# verfälschtes ergebniss da in der ausgabe neben den namen auch andere informationen stehen die vektorisiert wurden
# Erkentniss ist hier schon einmal dass je länger der Vektor (Vector_size) desto schwierig wird es ähnliche Namen zu finden/matchen

[('Maria Nikolaevna PRUSAKOVA   Member of the State Duma', 0.9089804887771606),
 ('Gulbachor ISMAILOVA', 0.895755410194397),
 ('Muhammad Ibrahim Al-Shaar', 0.8933694958686829),
 ('Abu Aisyah  Remark: low quality a.k.a.', 0.8718813061714172),
 ('Alina Maratovna KABAEVA   Chairwoman of the Board of Directors of the National Media Group (NMG)  Remark:\nAssociated  entity: National Media Group (NMG)',
  0.8669117093086243)]

### Using a pretrained Model for Word Embedding (Glove or Fastext)
  - Model Glove ( https://nlp.stanford.edu/projects/glove/)
  - Model FastText 

In [63]:
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors

# Download the model
glove_file = join('glove.6B.100d.txt')
tmp_file = get_tmpfile("test_word2vec.txt")

# Load the model
model = KeyedVectors.load_word2vec_format(glove_file)


ValueError: invalid literal for int() with base 10: 'the'

In [ ]:
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
import numpy as np
import os
GLOVE_DIR = 'Path to directory containing glove txt file'

embeddings_index = {}
f = open(os.path. join (GLOVE_DIR, 'glove.68.100d.txt'), encoding="utf8")
for line in f:
    values = line.split()
    word = values [0]
    coefs = np.asarray(values [1:], dtype='float32')
    embeddings_index[word] = coefs
f. close ()

In [ ]:
# Generate the word embedding for the word "cat"
word_embedding = model['cat']

# Print the word embedding
print(word_embedding)


In [ ]:
from scipy import spatial

# Generate the word embeddings for the words "cat" and "dog"
word1_embedding = model['cat']
word2_embedding = model['dog']

# Calculate the cosine similarity between the two word embeddings
similarity = 1 - spatial.distance.cosine(word1_embedding, word2_embedding)

# Print the similarity
print(similarity)


In [21]:
import numpy as np
import matplotlib 
from matplotlib import pyplot as plt
from sklearn.manifold import TSNE

In [22]:
# test for visualization 

def display_closestwords_tsnescatterplot(model, word, size):
    
    arr = np.empty((0,size), dtype='f')
    word_labels = [word]
    close_words = model.wv.similar_by_word(word)
    arr = np.append(arr, np.array([model.wv[word]]), axis=0)
    for wrd_score in close_words:
        wrd_vector = model.wv[wrd_score[0]]
        word_labels.append(wrd_score[0])
        arr = np.append(arr, np.array([wrd_vector]), axis=0)
        
        tsne = TSNE(n_components=2, random_state=0, perplexity=4)
        np.set_printoptions(suppress=True)
        Y = tsne.fit_transform(arr)
        x_coords = Y[:, 0]
        y_coords = Y[:, 1]
        plt.scatter(x_coords, y_coords)
    for label, x, y in zip(word_labels, x_coords, y_coords):
        plt.annotate(label, xy=(x, y), xytext=(0, 0), textcoords='offset points')
        plt.xlim(x_coords.min()+0.00005, x_coords.max()+0.00005)
        plt.ylim(y_coords.min()+0.00005, y_coords.max()+0.00005)
        plt.show()
display_closestwords_tsnescatterplot(model, 'Khalifa', 5) 

ValueError: perplexity must be less than n_samples

In [ ]:
import gensim
from gensim.models import Word2Vec
from sklearn.decomposition import PCA
from nltk.tokenize import sent_tokenize, word_tokenize
from matplotlib import pyplot
 
# #Preprocessing
# data = dbn_df['Identity information'] = dbn_df['Identity information'].apply(lambda content: [x for x in content.split(';')]) #Translating 'Identity information' into list object

# sentences_tokenized = []
# for i in data:
#     print(i)
#     # s = i.lower
#     # myPunc = '!"#%&$()*+-./:;,<=>?@[\\]^_`{|}~'
#     # s = s.translate(s.maketrans(myPunc, ' '*len(myPunc)))
#     # s = s.translate(s.maketrans(string.digits, ' '*len(string.digits)))
#     # s = re.sub('\s+',' ',s)
#     # print(s)
#     # for sentence in s:    
#     #     tokens = sentence.split(' ')
#     #     sentences_tokenized.append(tokens)


